<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Exact Wald `GiRaFFEfood` Initial Data for `GiRaFFE`

## Author: Zach Etienne & Patrick Nelson
### Formatting improvements courtesy Brandon Clark

[comment]: <> (Abstract: TODO)

### NRPy+ Source Code for this module: [GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Exact_Wald.py](../edit/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Exact_Wald.py)

**Notebook Status:** <font color='green'><b> Validated </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation1). The initial data has validated against the original `GiRaFFE`, as documented [here](Tutorial-Start_to_Finish_UnitTest-GiRaFFEfood_NRPy.ipynb).

## Introduction: 
With the `GiRaFFE` evolution thorn constructed, we now need to "feed" our giraffe with initial data to evolve. There are several different choices of initial data we can use here; here, we will only be implementing the "Exact Wald" initial data, given by Table 3 in [the original paper](https://arxiv.org/pdf/1704.00599.pdf):
\begin{align}
A_{\phi} &= \frac{C_0}{2} r^2 \sin^2 \theta \\
E_{\phi} &= 2 M C_0 \left( 1+ \frac {2M}{r} \right)^{-1/2} \sin^2 \theta \\
\end{align}
(the unspecified components are set to 0). Here, $C_0$ is a constant that we will set to $1$ in our simulations. Now, to use this initial data scheme, we need to transform the above into the quantities actually tracked by `GiRaFFE` and HydroBase: $A_i$, $B^i$, $\tilde{S}_i$, $v^i$, and $\Phi$. Of these quantities, $\texttt{GiRaFFEfood}$ will only set $A_i$, $v^i$, and $\Phi=0$, then call a separate function to calculate  $\tilde{S}_i$; `GiRaFFE` itself will call a function to set $B^i$ before the time-evolution begins. This can be done with eqs. 16 and 18, here given in that same order:
\begin{align}
v^i &= \alpha \frac{\epsilon^{ijk} E_j B_k}{B^2} -\beta^i \\
B^i &= \frac{[ijk]}{\sqrt{\gamma}} \partial_j A_k \\
\end{align}
In the simulations, $B^i$ will be calculated numerically from $A_i$; however, it will be useful to analytically calculate $B^i$ to use calculating the initial $v^i$.

This module requires the use of the NRPy+ [Shifted Kerr-Schild initial data module](../Tutorial-ADM_Initial_Data-ShiftedKerrSchild.ipynb)

<a id='toc'></a>

# Table of Contents:
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Import core NRPy+ modules and set NRPy+ parameters
1. [Step 2](#set_aphi_ephi): Set the vectors $A_{\phi}$ and $E_{\phi}$ in Spherical coordinates 
1. [Step 3](#jacobian): Use the Jacobian matrix to transform the vectors to Cartesian coordinates
1. [Step 4](#vi): Calculate $v^i$ from $A_i$ and $E_i$
1. [Step 5](#code_validation1): Code Validation against `GiRaFFEfood_NRPy.GiRaFFEfood_NRPy` NRPy+ Module
1. [Step 6](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Import core NRPy+ modules and set NRPy+ parameters \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Here, we will import the NRPy+ core modules, set the reference metric to Cartesian, and set commonly used NRPy+ parameters. We will also set up a parameter to determine what initial data is set up, although it won't do much yet.



In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Step 0.a: Import the NRPy+ core modules and set the reference metric to Cartesian
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support

import reference_metric as rfm
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()

# Step 1a: Set commonly used parameters.
thismodule = "GiRaFFEfood_NRPy"

# KerrSchild_radial_shift = par.Cparameters("REAL",thismodule,"KerrSchild_radial_shift",0.4) # Default value for ExactWald
KerrSchild_radial_shift = sp.symbols("KerrSchild_radial_shift")
M = sp.symbols("M") # Black hole mass

<a id='set_aphi_ephi'></a>

# Step 2: Set the vectors $A_{\phi}$ and $E_{\phi}$ in Spherical coordinates \[Back to [top](#toc)\]
$$\label{set_aphi_ephi}$$

We will first build the fundamental vectors $A_i$ and $E_i$ in spherical coordinates (see [Table 3](https://arxiv.org/pdf/1704.00599.pdf)). Note that we use reference_metric.py to set $r$ and $\theta$ in terms of Cartesian coordinates; this will save us a step later when we convert to Cartesian coordinates. Since $C_0 = 1$,
\begin{align}
A_{\phi} &= \frac{1}{2} r^2 \sin^2 \theta \\
E_{\phi} &= 2 M \left( 1+ \frac {2M}{r} \right)^{-1/2} \sin^2 \theta. \\
\end{align}
While we have $E_i$ set as a variable in NRPy+, note that the final C code won't store these values.



In [2]:
# Step 2: Set the vectors A and E in Spherical coordinates

r     = rfm.xxSph[0] + KerrSchild_radial_shift # We are setting the data up in Shifted Kerr-Schild coordinates
theta = rfm.xxSph[1]

# Initialize all components of A and E in the *spherical basis* to zero
ASphD = ixp.zerorank1()
ESphD = ixp.zerorank1()
ASphD[2] = (r * r * sp.sin(theta)**2)/2
ESphD[2] = 2 * M * sp.sin(theta)**2 / sp.sqrt(1+2*M/r)

<a id='jacobian'></a>

# Step 3: Use the Jacobian matrix to transform the vectors to Cartesian coordinates \[Back to [top](#toc)\]
$$\label{jacobian}$$

Now, we will use the coordinate transformation definitions provided by reference_metric.py to build the Jacobian 

$$ 
\frac{\partial x_{\rm Sph}^j}{\partial x_{\rm Cart}^i},
$$ 

where $x_{\rm Sph}^j \in \{r,\theta,\phi\}$ and $x_{\rm Cart}^i \in \{x,y,z\}$. We would normally compute its inverse, but since none of the quantities we need to transform have upper indices, it is not necessary. Then, since both $A_i$ and $E_i$ have one lower index, both will need to be multiplied by the Jacobian:

\begin{align}
A_i^{\rm Cart} &= A_j^{\rm Sph} \frac{\partial x_{\rm Sph}^j}{\partial x_{\rm Cart}^i} \\
{\rm and\ }E_i^{\rm Cart} &= E_j^{\rm Sph} \frac{\partial x_{\rm Sph}^j}{\partial x_{\rm Cart}^i}.
\end{align}



In [3]:
# Step 3: Use the Jacobian matrix to transform the vectors to Cartesian coordinates.
drrefmetric__dx_0UDmatrix = sp.Matrix([[sp.diff(rfm.xxSph[0],rfm.xx[0]), sp.diff(rfm.xxSph[0],rfm.xx[1]), sp.diff(rfm.xxSph[0],rfm.xx[2])],
                                       [sp.diff(rfm.xxSph[1],rfm.xx[0]), sp.diff(rfm.xxSph[1],rfm.xx[1]), sp.diff(rfm.xxSph[1],rfm.xx[2])],
                                       [sp.diff(rfm.xxSph[2],rfm.xx[0]), sp.diff(rfm.xxSph[2],rfm.xx[1]), sp.diff(rfm.xxSph[2],rfm.xx[2])]])
#dx__drrefmetric_0UDmatrix = drrefmetric__dx_0UDmatrix.inv() # We don't actually need this in this case.

AD = ixp.zerorank1()
ED = ixp.zerorank1()

for i in range(3):
    for j in range(3):
        AD[i] = drrefmetric__dx_0UDmatrix[(j,i)]*ASphD[j]
        ED[i] = drrefmetric__dx_0UDmatrix[(j,i)]*ESphD[j]

#Step 4: Declare the basic spacetime quantities
alpha   = sp.symbols("alpha",real=True)
betaU   = ixp.declarerank1("betaU",DIM=3)
gammaDD = ixp.declarerank2("gammaDD", "sym01",DIM=3)

import GRHD.equations as GRHD
GRHD.compute_sqrtgammaDET(gammaDD)

<a id='vi'></a>

# Step 4: Calculate $v^i$ from $A_i$ and $E_i$ \[Back to [top](#toc)\]
$$\label{vi}$$

We will now find the magnetic field using equation 18 in [the original paper](https://arxiv.org/pdf/1704.00599.pdf) $$B^i = \frac{[ijk]}{\sqrt{\gamma}} \partial_j A_k. $$ We will need the metric quantites: the lapse $\alpha$, the shift $\beta^i$, and the three-metric $\gamma_{ij}$. We will also need the Levi-Civita symbol. 

In [4]:
# Step 4: Calculate v^i from A_i and E_i
# Step 4a: Calculate the magnetic field B^i

GRHD.compute_sqrtgammaDET(gammaDD)
LeviCivitaTensorUUU = ixp.LeviCivitaTensorUUU_dim3_rank3(GRHD.sqrtgammaDET)

# For the initial data, we can analytically take the derivatives of A_i
ADdD = ixp.zerorank2()
for i in range(3):
    for j in range(3):
        ADdD[i][j] = sp.simplify(sp.diff(AD[i],rfm.xxCart[j]))

BU = ixp.zerorank1()
for i in range(3):
    for j in range(3):
        for k in range(3):
            BU[i] += LeviCivitaTensorUUU[i][j][k] * ADdD[k][j]

We will now build the initial velocity using equation 152 in [this paper,](https://arxiv.org/pdf/1310.3274v2.pdf) cited in the original `GiRaFFE` code: $$ v^i = \alpha \frac{\epsilon^{ijk} E_j B_k}{B^2} -\beta^i. $$ 
However, our code needs the Valencia 3-velocity while this expression is for the drift velocity. So, we will need to transform it to the Valencia 3-velocity using the rule $\bar{v}^i = \frac{1}{\alpha} \left(v^i +\beta^i \right)$.
Thus, $$\bar{v}^i = \frac{\epsilon^{ijk} E_j B_k}{B^2}$$

In [5]:
# Step 4b: Calculate B^2 and B_i
# B^2 is an inner product defined in the usual way:
B2 = sp.sympify(0)
for i in range(3):
    for j in range(3):
        B2 += gammaDD[i][j] * BU[i] * BU[j]

# Lower the index on B^i
BD = ixp.zerorank1()
for i in range(3):
    for j in range(3):
        BD[i] += gammaDD[i][j] * BU[j]

# Step 4c: Calculate the Valencia 3-velocity
ValenciavU = ixp.zerorank1()
for i in range(3):
    for j in range(3):
        for k in range(3):
            ValenciavU[i] += LeviCivitaTensorUUU[i][j][k]*ED[j]*BD[k]/B2

<a id='code_validation1'></a>

# Step 5: Code Validation against `GiRaFFEfood_NRPy.GiRaFFEfood_NRPy` NRPy+ module  \[Back to [top](#toc)\]
$$\label{code_validation1}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for the `GiRaFFE` Exact Wald initial data equations  we intend to use between
1. this tutorial and 
2. the NRPy+ [GiRaFFEfood_NRPy.GiRaFFEfood_NRPy_Exact_Wald](../edit/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Exact_Wald.py) module.


In [6]:
import GiRaFFEfood_NRPy.GiRaFFEfood_NRPy_Exact_Wald as gfho
gfho.GiRaFFEfood_NRPy_Exact_Wald(gammaDD,M,KerrSchild_radial_shift)

def consistency_check(quantity1,quantity2,string):
    if quantity1-quantity2==0:
        print(string+" is in agreement!")
    else:
        print(string+" does not agree!")
        sys.exit(1)

print("Consistency check between GiRaFFEfood_NRPy tutorial and NRPy+ module:")

for i in range(3):
    consistency_check(ValenciavU[i],gfho.ValenciavU[i],"ValenciavU"+str(i))
    consistency_check(AD[i],gfho.AD[i],"AD"+str(i))

Consistency check between GiRaFFEfood_NRPy tutorial and NRPy+ module:
ValenciavU0 is in agreement!
AD0 is in agreement!
ValenciavU1 is in agreement!
AD1 is in agreement!
ValenciavU2 is in agreement!
AD2 is in agreement!


<a id='latex_pdf_output'></a>

# Step 6: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFEfood_NRPy.pdf](Tutorial-GiRaFFEfood_NRPy.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [7]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GiRaFFEfood_NRPy_Exact_Wald")

Created Tutorial-GiRaFFEfood_NRPy_Exact_Wald.tex, and compiled LaTeX file
    to PDF file Tutorial-GiRaFFEfood_NRPy_Exact_Wald.pdf
